David Guo

# HW 3

I follow the linked python book, using a random forest regression as a classifier, with 100 trees, following a unigram model for text classification. I use LIME reader with 6 features. I identify and remove the feature with the lowest (negative) score for text that was classified positive in the test data and remove the feature with the highest (positive) score for text that was classified negative (due to long run time, I only did this for about one-third of the results). I remove these words from the training data and re-train the classifier. The F-score increased from 0.52 to 0.77. (For example, in the test set, entry 83, which was positive, I removed the word "Britney", which was the highest negative score from the LIME feataures.)

I attempted the same approach with an SVM classifier (SGD training with hinge loss), with l2 penalty. The baseline performs well with an F-score of 0.863, but it did not have a predict_proba function, so I couldn't test it with LIME.

So I tried a linear classifier with a log loss using SGD training. The baseline performs well with an F-Score of 0.83. Instead of finding bad features for 1/3 of the results, I sampled 250 of the text in the test set. Unfortunately, this is probably too small as the F-score was smaller at about 0.82. I increased the same to 1000, which yielded 485 bad features and increased the F-score to 0.8325. This is what I use for the private leaderboard scoring.

When I tried the same log loss classifier the sublinear term frequency in the vectorizer, the baseline yielded a higher F-score of 0.85, but a lower score of 0.82 after LIME with about 350 bad features removed. It may be that longer reviews do have more important features so log-transforming the term frequency might only be useful if we have more than 6 features.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
import pandas
trainset=pandas.read_table("./sentiment.train.tsv", quoting=3, quotechar="\"")
testset=pandas.read_table("./sentiment.test-data.tsv", quoting=3, quotechar="\"")

In [4]:
import lime
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

In [72]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(trainset['Text'])
test_vectors = vectorizer.transform(testset['Text'])


In [6]:
target=np.array(trainset['Rating'])
target

array([0, 0, 1, ..., 0, 1, 1])

In [6]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=100)

In [20]:
rf.fit(train_vectors, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
pred = rf.predict(test_vectors)

In [30]:
out=pandas.DataFrame({'Id': testset['Id'], 'Rating': pred})
out

,Id,Rating
0,36001,1
1,36002,1
2,36003,1
3,36004,1
4,36005,1
5,36006,1
6,36007,1
7,36008,1
8,36009,1
9,36010,1


In [31]:
out.to_csv('./output.csv', index=False)

In [ ]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [ ]:
#print(c.predict_proba([testset['Text'][0]]))

In [24]:
from lime.lime_text import LimeTextExplainer
class_names=['negative','positive']
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
#idx = 83
#exp = explainer.explain_instance(testset['Text'][idx], c.predict_proba, num_features=6)
#print('Document id: %d' % idx)
#print('Probability(positive) =', c.predict_proba([testset['Text'][idx]])[0,1])
#print('True class: %s' % class_names[pred[idx]])

In [11]:
#min(exp.as_list()[1])

NameError: name 'exp' is not defined

In [160]:
badfeat=[]

for idx in range(0,len(testset)):
    exp=explainer.explain_instance(testset['Text'][idx], c.predict_proba, num_features=6)
    if pred[idx]==1: #if positive
        badfeat.append(exp.as_list()[np.argmin([x[1] for x in exp.as_list()])][0]) #remove largest negative score feature
    else: #if negative
        badfeat.append(exp.as_list()[np.argmax([x[1] for x in exp.as_list()])][0]) #remove largest positive score feature

/Users/davidguo/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


KeyboardInterrupt: 

In [9]:
badfeatuniq=set(badfeat)
#badfeatuniq

{'34',
 '50',
 'All',
 'BUT',
 'BUY',
 'Beatles',
 'Brenda',
 'Britney',
 'But',
 'CD',
 'Carly',
 'Classic',
 'Come',
 'DIDN',
 'DON',
 'DONT',
 'Do',
 'Don',
 'EARL',
 'Every',
 'Excellent',
 'GOT',
 'Great',
 'Highly',
 'Hogarth',
 'How',
 'It',
 'Italia',
 'JUST',
 'Jazz',
 'Jumbo',
 'LOVE',
 'La',
 'Love',
 'Lucassen',
 'MTV',
 'MY',
 'Maybe',
 'Mercy',
 'NOT',
 'No',
 'OF',
 'Oh',
 'Rainbows',
 'Relf',
 'SAY',
 'SHOULD',
 'Save',
 'Sorry',
 'Stay',
 'The',
 'This',
 'Too',
 'Unfortunately',
 'Very',
 'WHAT',
 'WILLIAMS',
 'Well',
 'Where',
 'Why',
 'Wild',
 'Wonderful',
 'You',
 'about',
 'actually',
 'addition',
 'after',
 'album',
 'albums',
 'alot',
 'also',
 'always',
 'amazing',
 'an',
 'and',
 'annoying',
 'anymore',
 'anything',
 'are',
 'as',
 'at',
 'attempt',
 'attempts',
 'average',
 'away',
 'awesome',
 'awful',
 'back',
 'bad',
 'barely',
 'bass',
 'be',
 'beautiful',
 'beautifully',
 'because',
 'before',
 'best',
 'better',
 'big',
 'bland',
 'bluesy',
 'bonus',
 '

In [10]:
import pickle

#with open('outfile', 'wb') as fp:
#    pickle.dump(badfeat, fp)
    
with open ('outfile', 'rb') as fp:
    badfeat = pickle.load(fp)

with open('badfeatuniq', 'wb') as fp:
    pickle.dump(badfeatuniq, fp)

In [11]:
newtrain=[' '.join(w for w in text.split() if w.lower() not in badfeatuniq)
         for text in trainset['Text']
         ]
newtest=[' '.join(w for w in text.split() if w.lower() not in badfeatuniq)
         for text in testset['Text']
         ]

#https://stackoverflow.com/questions/3510846/how-to-remove-list-of-words-from-a-list-of-strings

In [20]:
trainnew_vectors = vectorizer.fit_transform(newtrain)
testnew_vectors = vectorizer.transform(newtest)

In [21]:
rf.fit(trainnew_vectors, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
newpred=rf.predict(testnew_vectors)

In [23]:
out2=pandas.DataFrame({'Id': testset['Id'], 'Rating': newpred})
#out2

,Id,Rating
0,36001,1
1,36002,0
2,36003,0
3,36004,1
4,36005,0
5,36006,1
6,36007,0
7,36008,1
8,36009,0
9,36010,0


In [25]:
out2.to_csv('./output2.csv', index=False)

In [12]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm

In [54]:
logclf=SGDClassifier(loss='log', penalty='l2',
                                            alpha=1e-4, random_state=42,
                                            max_iter=100, tol=None)

In [55]:
logclf.fit(train_vectors,target)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=100, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [56]:
log_pred=logclf.predict(test_vectors)

In [57]:
log_out=pandas.DataFrame({'Id': testset['Id'], 'Rating': log_pred})
#log_out

,Id,Rating
0,36001,1
1,36002,0
2,36003,0
3,36004,1
4,36005,0
5,36006,1
6,36007,1
7,36008,1
8,36009,0
9,36010,0


In [58]:
log_out.to_csv('./out_log_sublin.csv', index=False)

In [59]:
c_log = make_pipeline(vectorizer, logclf)

In [76]:
import random
random.seed(8329)
badsamp=random.sample(range(0,len(newtest)),1000)

In [61]:
badfeatlog=[]

for idx in badsamp:
    exp=explainer.explain_instance(newtest[idx], c_log.predict_proba, num_features=6)
    if log_pred[idx]==1: #if positive
        badfeatlog.append(exp.as_list()[np.argmin([x[1] for x in exp.as_list()])][0]) #remove largest negative score feature
    else: #if negative
        badfeatlog.append(exp.as_list()[np.argmax([x[1] for x in exp.as_list()])][0]) #remove largest positive score feature

/Users/davidguo/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [62]:
import pickle

with open('badfeatlog', 'wb') as fp:
    pickle.dump(badfeatlog, fp)
    
with open ('badfeatlog', 'rb') as fp:
    badfeatlog = pickle.load(fp)

In [71]:
badfeatloguniq=set(badfeatlog)
#badfeatloguniq

{'10',
 '34',
 '70s',
 'Achtung',
 'Adams',
 'All',
 'Animals',
 'BB',
 'BOC',
 'Baba',
 'Band',
 'Beck',
 'Bennett',
 'Blood',
 'Blues',
 'British',
 'But',
 'Buy',
 'CD',
 'Cassandra',
 'Christmas',
 'Don',
 'EVERY',
 'Erasure',
 'Every',
 'Everybody',
 'Five',
 'GO',
 'God',
 'Goo',
 'Good',
 'Guster',
 'He',
 'Hip',
 'How',
 'However',
 'IF',
 'IT',
 'Idol',
 'If',
 'It',
 'Josh',
 'Jovi',
 'Keep',
 'Kelly',
 'Kertesz',
 'Kiss',
 'Limp',
 'Love',
 'Madame',
 'Maybe',
 'Metallica',
 'My',
 'NOT',
 'Neil',
 'Neverland',
 'Nickelback',
 'Norah',
 'Nothing',
 'One',
 'Ono',
 'Order',
 'Paul',
 'Private',
 'Requiem',
 'Rod',
 'Seriously',
 'Sorry',
 'Szeryng',
 'Tesla',
 'Thanks',
 'This',
 'Though',
 'Times',
 'Toto',
 'Trans',
 'Unfortunately',
 'Vida',
 'Wanna',
 'Where',
 'Whether',
 'Whitesnake',
 'Why',
 'YOU',
 'You',
 'addition',
 'album',
 'all',
 'alone',
 'along',
 'also',
 'am',
 'amazing',
 'and',
 'another',
 'anything',
 'arragements',
 'avoid',
 'awesome',
 'awful',
 'ba

In [66]:
newtrainlog=[' '.join(w for w in text.split() if w.lower() not in badfeatloguniq)
         for text in trainset['Text']
         ]
newtestlog=[' '.join(w for w in text.split() if w.lower() not in badfeatloguniq)
         for text in testset['Text']
         ]

#https://stackoverflow.com/questions/3510846/how-to-remove-list-of-words-from-a-list-of-strings

In [67]:
trainnewlog_vectors = vectorizer.fit_transform(newtrainlog)
testnewlog_vectors = vectorizer.transform(newtestlog)

In [68]:
logclf.fit(trainnewlog_vectors,target)
logprednew=logclf.predict(testnewlog_vectors)

In [69]:
log_out_new=pandas.DataFrame({'Id': testset['Id'], 'Rating': logprednew})

,Id,Rating
0,36001,1
1,36002,0
2,36003,0
3,36004,1
4,36005,0
5,36006,1
6,36007,1
7,36008,1
8,36009,0
9,36010,0


In [70]:
log_out_new.to_csv('./out_lognew750sublin.csv', index=False)

In [53]:
#vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False, sublinear_tf=True)
#train_vectors = vectorizer.fit_transform(trainset['Text'])
#test_vectors = vectorizer.transform(testset['Text'])

In [79]:
#svmmod=sklearn.svm.SVC(probability=True)

In [ ]:
svmmod.fit(train_vectors, target)

In [75]:
svmpred=svmmod.predict(test_vectors)
svm_out=pandas.DataFrame({'Id': testset['Id'], 'Rating': svmpred})
svm_out.to_csv('./out_svm.csv', index=False)

In [77]:
c=make_pipeline(vectorizer, svmmod)

In [ ]:
#badfeatsvm=[]
#
#for idx in badsamp:
#    exp=explainer.explain_instance(testset['Text'][idx], c.predict_proba, num_features=6)
#    if pred[idx]==1: #if positive
#        badfeatsvm.append(exp.as_list()[np.argmin([x[1] for x in exp.as_list()])][0]) #remove largest negative score feature
#    else: #if negative
#        badfeatsvm.append(exp.as_list()[np.argmax([x[1] for x in exp.as_list()])][0]) #remove largest positive score feature